In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:
data = pd.read_csv("D:\\Data\\NLP\\pradip\\toxic_commnets.csv",skiprows = range(1000, 159570), on_bad_lines='error', engine="python")
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data["toxic"].value_counts()

toxic
0    896
1    105
Name: count, dtype: int64

In [4]:
data = data[0:100]
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data["toxic"].value_counts()

toxic
0    86
1    14
Name: count, dtype: int64

In [6]:
#!pip install transformers -U
#!pip install accelerate

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

In [9]:
from accelerate import Accelerator
from transformers import TrainingArguments, Trainer

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [12]:
X = list(data["comment_text"])
y = list(data["toxic"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [13]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [15]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [16]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [17]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=10, training_loss=0.4575087547302246, metrics={'train_runtime': 559.2162, 'train_samples_per_second': 0.143, 'train_steps_per_second': 0.018, 'total_flos': 21048884428800.0, 'train_loss': 0.4575087547302246, 'epoch': 1.0})

In [19]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.41383132338523865,
 'eval_accuracy': 0.85,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_runtime': 13.3008,
 'eval_samples_per_second': 1.504,
 'eval_steps_per_second': 0.226,
 'epoch': 1.0}

In [20]:
np.set_printoptions(suppress=True)

In [21]:
text = "That was good point"
#text = "bad boy go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1526, -0.8409]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.7298, 0.2702]], grad_fn=<SoftmaxBackward0>)


array([[0.72977877, 0.2702212 ]], dtype=float32)

In [22]:
trainer.save_model("C:\\Users\\intel\\Projects\\NLP\\pradip\\custom_model\\Bert")

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [25]:
model_2 = BertForSequenceClassification.from_pretrained("C:\\Users\\intel\\Projects\\NLP\\pradip\\custom_model\\Bert")

In [26]:
text = "That was good point"
#text = "bad boy go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.72977877, 0.2702212 ]], dtype=float32)